### Decision Tree

In [211]:
from sklearn import datasets
iris = datasets.load_iris()


<class 'sklearn.utils.Bunch'>


In [150]:
import pandas as pd
data=pd.DataFrame({
    'sepal length':iris.data[:,0],
    'sepal width':iris.data[:,1],
    'petal length':iris.data[:,2],
    'petal width':iris.data[:,3],
    'species':iris.target
})
data.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
from sklearn.model_selection import train_test_split

X=data[['sepal length', 'sepal width', 'petal length', 'petal width']]  # Features
y=data['species']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% tes

### Comparision between DT and RF

In [280]:
wine_dataset = pd.read_csv('wine.csv', names=["Class","Alcohol", "Malic Acid", "Ash", "Alcalinity of ash","Magnesium","Total phenols",
                        "Flavanoids","Nonflavanoid phenols","Proanthocyanins","Color intensity","Hue",
                         "OD280/OD315 of diluted wines","Proline"])

print(wine_dataset.head())
X_wine = wine_dataset.loc[:,'Alcohol':]
Y_wine = wine_dataset.loc[:,'Class']



   Class  Alcohol  Malic Acid   Ash  Alcalinity of ash  Magnesium  \
0      1    14.23        1.71  2.43               15.6        127   
1      1    13.20        1.78  2.14               11.2        100   
2      1    13.16        2.36  2.67               18.6        101   
3      1    14.37        1.95  2.50               16.8        113   
4      1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
0             5.64  1.04                          3.92     1065  
1             4.38  1.05  

### DT classifier

In [281]:
from sklearn.model_selection import train_test_split

X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(X_wine, Y_wine, test_size=0.3) # 70% training and 30% test

tree = DecisionTreeClassifier()
tree.fit(X_train_wine, y_train_wine)

print(f'DT Model Accuracy: {tree.score(X_test_wine, y_test_wine)}')


DT Model Accuracy: 0.9259259259259259


### RF classifier

In [282]:
from sklearn.ensemble import RandomForestClassifier


rf_clf_wine=RandomForestClassifier(oob_score=True,bootstrap=True)


#Train the model using the training sets
rf_clf_wine.fit(X_train_wine,y_train_wine)
print(rf_clf_wine.predict([[13.45,3.7,2.6,23,111,1.7,.92,.43,1.46,10.68,.85,1.56,695]]))

print(f'RF Model Accuracy: {rf_clf_wine.score(X_test_wine, y_test_wine)}')


[3]
RF Model Accuracy: 0.9629629629629629


/usr/local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


### Choose best estimator

In [283]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    #'bootstrap': [True, False]
}




# Create the random search model
rs = RandomizedSearchCV(rf_clf_wine, param_grid)

rs.fit(X_train_wine,y_train_wine)

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=10,
                                                    n_jobs=Non

In [284]:
rs.best_params_


{'n_estimators': 161,
 'min_samples_split': 2,
 'max_leaf_nodes': 44,
 'max_features': 0.7,
 'max_depth': 12}

In [285]:
best_model = rs.best_estimator_

best_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=12, max_features=0.7, max_leaf_nodes=44,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=161,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [286]:
best_model.score(X_test_wine,y_test_wine)

0.9629629629629629

### Feature Importance

In [287]:
import pandas as pd
feature_imp = pd.Series(rf_clf_wine.feature_importances_,index=["Alcohol", "Malic Acid", "Ash", "Alcalinity of ash","Magnesium","Total phenols",
                        "Flavanoids","Nonflavanoid phenols","Proanthocyanins","Color intensity","Hue",
                         "OD280/OD315 of diluted wines","Proline"]).sort_values(ascending=False)
feature_imp

Proline                         0.246823
Flavanoids                      0.160172
Hue                             0.146897
Color intensity                 0.110790
Alcohol                         0.090138
OD280/OD315 of diluted wines    0.079006
Proanthocyanins                 0.049188
Ash                             0.037739
Total phenols                   0.034688
Magnesium                       0.015493
Alcalinity of ash               0.012823
Malic Acid                      0.008355
Nonflavanoid phenols            0.007888
dtype: float64

### Out-of-bag error

In [288]:
oob_error = 1 - rf_clf_wine.oob_score_
print(oob_error)


0.13709677419354838


### Random Forest for Regression

In [289]:
datasets = pd.read_csv('Random-Forest-Regression/Position_Salaries.csv')
print(datasets)
X = datasets.iloc[:, 1:2].values
y = datasets.iloc[:, 2].values
print(X,y)

            Position  Level   Salary
0   Business Analyst      1    45000
1  Junior Consultant      2    50000
2  Senior Consultant      3    60000
3            Manager      4    80000
4    Country Manager      5   110000
5     Region Manager      6   150000
6            Partner      7   200000
7     Senior Partner      8   300000
8            C-level      9   500000
9                CEO     10  1000000
[[ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]] [  45000   50000   60000   80000  110000  150000  200000  300000  500000
 1000000]


In [290]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 300, random_state = 0)
regressor.fit(X,y)

y_pred = regressor.predict([[34]])

print(y_pred)

[806666.66666667]
